Neural Networks and Deep Learning - 2nd Project

In [ ]:
%reset

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from keras import layers, models, Model
from sklearn.svm import SVC, LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.datasets import cifar10
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid


(xtr, ytr), (xte, yte) = cifar10.load_data() # Loading CIFAR-10 dataset
classes =  ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

Preprocessing the data

In [ ]:
Ntrain, Ntest = 10000, 10000 # Loading part of dataset
(xtrain, ytrain), (xtest, ytest) = (xtr[:Ntrain]/255.0, ytr[:Ntrain]), (xte[:Ntest]/255.0, yte[:Ntest])


x_train = xtrain.reshape(xtrain.shape[0], -1)
x_test = xtest.reshape(xtest.shape[0], -1)
ytrain = ytrain.flatten()  
ytest = ytest.flatten()

# PCA keeping 90% of information
pca = PCA(n_components=0.90)
xtrain_pca = pca.fit_transform(x_train)
xtest_pca = pca.transform(x_test)

# !! the file for the calculation of skewness, kurtosis and entropy statistics is contained in the 1st project submission (skew_kurt_entr_features.ipynb) !!
ske_stats_train = np.load('train_skew_kurt_entr.npy')[:Ntrain]
ske_stats_test = np.load('test_skew_kurt_entr.npy')[:Ntest]

# !! the file for the calculation of bispectrum statistics is contained in the 1st project submission (bispectrum_features.mlx) !!
bisp_stats_train = pd.read_excel('C:/Users/samag/OneDrive/Υπολογιστής/9o Εξάμηνο/Neural Networks - Deep Learning/Exercises/ex1/Matlab/train_bispectrum_stats_new.xlsx')[:Ntrain]
bisp_stats_test = pd.read_excel('C:/Users/samag/OneDrive/Υπολογιστής/9o Εξάμηνο/Neural Networks - Deep Learning/Exercises/ex1/Matlab/test_bispectrum_stats_new.xlsx')[:Ntest]


xtrain_in = np.hstack([xtrain_pca, ske_stats_train, bisp_stats_train])
xtest_in = np.hstack([xtest_pca, ske_stats_test, bisp_stats_test])


scaler = StandardScaler()
xtrain_in = scaler.fit_transform(xtrain_in)
xtest_in = scaler.transform(xtest_in)

Tuning SVMs with various parameters 

In [ ]:
# Parameters
C_values = [0.1, 1, 10]
gamma_values = [0.001, 0.01, 0.1]
degree_values = [2, 3, 4]


linear_time = []
linear_parameters = []
polynomial_time = []
polynomial_parameters = []
rbf_time = []
rbf_parameters = []

results = {
    "Linear": [],
    "Polynomial": [],
    "RBF": []
}

# Linear SVM
for i in C_values:
    print(f"\nTraining Linear SVM with C={i}")


    linear_st = time.time() # start time (st)

    linearSVC = OneVsRestClassifier(LinearSVC(C=i, random_state=42, max_iter=10000))
    linearSVC.fit(xtrain_in, ytrain)

    linear_et = time.time() # end time (et)
    linear_rt= linear_et - linear_st # response time (rt)

    linear_time.append(linear_rt)
    linear_parameters.append(f"C={i}")
            
    print(f"Training Time: {linear_rt:.2f} seconds")
            
    linear_tr_predictions = linearSVC.predict(xtrain_in) # train accuracy
    train_accuracy = accuracy_score(ytrain, linear_tr_predictions)
    print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
            

    linear_te_predictions = linearSVC.predict(xtest_in) # test accuracy
    test_accuracy = accuracy_score(ytest, linear_te_predictions)
    print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

    results["Linear"].append((f"C={i}", train_accuracy*100, test_accuracy*100))

   
    linear_cm = confusion_matrix(ytest, linear_te_predictions)  # Confusion Matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=linear_cm, display_labels=classes)
    disp.plot(cmap="Blues", values_format="d")
    plt.title(f"Confusion Matrix (Linear, C={i})")
    plt.xlabel('Predicted Class')
    plt.ylabel('True Class')
    plt.xticks(rotation=90)
    plt.show()    


# plot of training times for polynomial kernel
plt.barh(linear_parameters, linear_time, color='skyblue')
plt.xlabel("Training Time (seconds)")
plt.ylabel("C")
plt.title("Training Time for Linear SVM Models")
plt.tight_layout()
plt.show()


# Polynomial SVM
for i in C_values:
    for j in degree_values:
            print(f"\nTraining Polynomial SVM with C={i}, degree={j}")
            
            
            polynomial_st = time.time() # start time (st)
            
            polynomialSVC = OneVsRestClassifier(SVC(kernel="poly", C=i, degree=j))
            polynomialSVC.fit(xtrain_in, ytrain)
            
            polynomial_et = time.time() # end time (et)
            polynomial_rt = polynomial_et - polynomial_st # response time (rt)

            polynomial_time.append(polynomial_rt)
            polynomial_parameters.append(f"C={i}, degree={j}")
            
            print(f"Training Time: {polynomial_rt:.2f} seconds")
            
            polynomial_tr_predictions = polynomialSVC.predict(xtrain_in) # train accuracy
            train_accuracy = accuracy_score(ytrain, polynomial_tr_predictions)
            print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
            
            polynomial_te_predictions = polynomialSVC.predict(xtest_in) # test accuracy
            test_accuracy = accuracy_score(ytest, polynomial_te_predictions)
            print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

            results["Polynomial"].append((f"C={i}, degree={j}", train_accuracy* 100, test_accuracy* 100))
            
            # Confusion Matrix
            polynomial_cm = confusion_matrix(ytest, polynomial_te_predictions)
            disp = ConfusionMatrixDisplay(confusion_matrix=polynomial_cm, display_labels=classes)
            disp.plot(cmap="Blues", values_format="d")
            plt.title(f"Confusion Matrix (Polynomial, C={i}, degree={j})")
            plt.xlabel('Predicted Class')
            plt.ylabel('True Class')
            plt.xticks(rotation=90)
            plt.show()

# plot of training times for polynomial kernel
plt.barh(polynomial_parameters, polynomial_time, color='skyblue')
plt.xlabel("Training Time (seconds)")
plt.ylabel("C, Degree Combinations")
plt.title("Training Time for Polynomial SVM Models")
plt.tight_layout()
plt.show()


# RBF SVM
for i in C_values:
    for j in gamma_values:
        print(f"\nTraining SVM with C={i}, gamma={j}")
        
        rbf_st = time.time() # start time (st)
        
        rbfSVC = OneVsRestClassifier(SVC(kernel="rbf", C=i, gamma=j))
        rbfSVC.fit(xtrain_in, ytrain)
        
        rbf_et = time.time() # end time (et)
        rbf_rt = rbf_et - rbf_st # response time (rt)

        rbf_time.append(rbf_rt)
        rbf_parameters.append(f"C={i}, \u03b3={j}")  
        
        print(f"Training Time: {rbf_rt:.2f} seconds")
        
        rbf_tr_predictions = rbfSVC.predict(xtrain_in) # train accuracy
        train_accuracy = accuracy_score(ytrain, rbf_tr_predictions)
        print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
        
        rbf_te_predictions = rbfSVC.predict(xtest_in) # test accuracy
        test_accuracy = accuracy_score(ytest, rbf_te_predictions)
        print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

        results["RBF"].append((f"C={i}, gamma={j}", train_accuracy* 100, test_accuracy* 100))
        
        # Confusion Matrix
        rbf_cm = confusion_matrix(ytest, rbf_te_predictions)
        disp = ConfusionMatrixDisplay(confusion_matrix=rbf_cm, display_labels=classes)
        disp.plot(cmap="Blues", values_format="d")
        plt.title(f"Confusion Matrix (RBF, C={i}, \u03b3={j})")
        plt.xlabel('Predicted Class')
        plt.ylabel('True Class')
        plt.xticks(rotation=90)
        plt.show()

# plot of training times of RBF SVM
plt.barh(rbf_parameters, rbf_time, color='skyblue')
plt.xlabel("Training Time (seconds)")
plt.ylabel("C, \u03b3 Combinations")
plt.title("Training Time for RBF Kernel SVM Models")
plt.tight_layout()
plt.show()


# the following function for the plot of accuracies is provided by CHAT-GPT
def plot_accuracies(results, title):
    parameter_combinations = [res[0] for res in results]
    train_accuracies = [res[1] for res in results]
    test_accuracies = [res[2] for res in results]

    plt.figure(figsize=(12, 6))
    plt.plot(range(len(parameter_combinations)), train_accuracies, marker="o", label="Training Accuracy")
    plt.plot(range(len(parameter_combinations)), test_accuracies, marker="s", label="Test Accuracy")

    plt.ylim(0, 100)
    plt.xlabel("Parameter Combination")
    plt.ylabel("Accuracy (%)")
    plt.title(title)
    plt.xticks(range(len(parameter_combinations)), parameter_combinations, rotation=45, ha="right")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()



plot_accuracies(results["Linear"], "Linear SVM: Training and Test Accuracies")
plot_accuracies(results["Polynomial"], "Polynomial SVM: Training and Test Accuracies")
plot_accuracies(results["RBF"], "RBF SVM: Training and Test Accuracies")


Train RBF SVC with the whole train set for C=0.1 and gamma=0.001 (I will take only the first one finally, c=0.1 and gamma=0.001 )

In [5]:
xtr, xte = xtr/255.0, xte/255.0


xtr = xtr.reshape(xtr.shape[0], -1)
xte = xte.reshape(xte.shape[0], -1)
ytr = ytr.flatten() 
yte = yte.flatten()

# PCA keeping 90% of information
pca2 = PCA(n_components=0.90)
xtr_pca = pca2.fit_transform(xtr)
xte_pca = pca2.transform(xte)

ske_stats_tr = np.load('train_skew_kurt_entr.npy')
ske_stats_te = np.load('test_skew_kurt_entr.npy')

bisp_stats_tr = pd.read_excel('C:/Users/samag/OneDrive/Υπολογιστής/9o Εξάμηνο/Neural Networks - Deep Learning/Exercises/ex1/Matlab/train_bispectrum_stats_new.xlsx')
bisp_stats_te = pd.read_excel('C:/Users/samag/OneDrive/Υπολογιστής/9o Εξάμηνο/Neural Networks - Deep Learning/Exercises/ex1/Matlab/test_bispectrum_stats_new.xlsx')

# Combine PCA features with statistics
xtr_in = np.hstack([xtr_pca, ske_stats_tr, bisp_stats_tr])
xte_in = np.hstack([xte_pca, ske_stats_te, bisp_stats_te])


Train RBF SVM with C=0.1 and gammma=0.001 with the whole train set

In [ ]:
C, gamma = 0.1, 0.001
print(f"\nTraining SVM with C={C}, gamma={gamma} for the whole train set")

scaler = StandardScaler()
xtr_ins = scaler.fit_transform(xtr_in) 
xte_ins = scaler.transform(xte_in)


rbfSVC2_st = time.time() # start time (st)
        
rbfSVC2 = OneVsRestClassifier(SVC(kernel="rbf", C=C, gamma=gamma))
rbfSVC2.fit(xtr_ins, ytr)
        
rbfSVC2_et = time.time() # end time (et)
rbfSVC2_rt = rbfSVC2_et - rbfSVC2_st # response time (rt)


print(f"Training Time: {rbfSVC2_rt:.2f} seconds")
        

rbfSVC2_tr_predictions = rbfSVC2.predict(xtr_ins) 
train_accuracy = accuracy_score(ytr, rbfSVC2_tr_predictions) # train accuracy
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
        
rbfSVC2_te_predictions = rbfSVC2.predict(xte_ins)
test_accuracy = accuracy_score(yte, rbfSVC2_te_predictions) # test accuracy
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

        

rbfSVC2_cm = confusion_matrix(yte, rbfSVC2_te_predictions) # Confusion Matrix
disp = ConfusionMatrixDisplay(confusion_matrix=rbfSVC2_cm, display_labels=classes)
disp.plot(cmap="Blues", values_format="d")
plt.title(f"Confusion Matrix (RBF, C={C}, \u03b3={gamma})")
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.xticks(rotation=90)
plt.show()

MLP with one hidden layer and Hinge loss

In [ ]:
ytr_oh = tf.keras.utils.to_categorical(ytr, 10) #one hot encoding
yte_oh = tf.keras.utils.to_categorical(yte, 10) #one hot encoding


mlpHinge_input2 = layers.Input(shape=(xtr_in.shape[1],), name='mlpHinge_in')
mlp2=models.Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='softmax')
])            
mlpHinge_output2 = mlp2(mlpHinge_input2)  

mlpHinge2 = Model(inputs=mlpHinge_input2, outputs=mlpHinge_output2)  


mlpHinge2.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=0.01),
              loss='hinge', 
              metrics=['accuracy'])


mlpHinge_st = time.time() # start time (st)

mlpHinge_history2 = mlpHinge2.fit(xtr_in, ytr_oh, 
                    batch_size=128, 
                    epochs=30, 
                    )
mlpHinge_et = time.time() # end time (et)
mlpHinge_rt = mlpHinge_et-mlpHinge_st # response time (rt)



test_loss, test_accuracy = mlpHinge2.evaluate(xte_in, yte_oh)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy*100:.4f}")



mlpHinge2_predictions = mlpHinge2.predict(xte_in)
mlpHinge2_predictions = np.argmax(mlpHinge2_predictions, axis=1)
mlpHinge2_cm = confusion_matrix(yte, mlpHinge2_predictions)

disp = ConfusionMatrixDisplay(confusion_matrix=mlpHinge2_cm, display_labels=classes)
disp.plot(cmap='Blues', xticks_rotation=90)  
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.title("Confusion Matrix")
plt.show()



# Plots:
# accuracy
plt.subplot(1, 2, 1) 
plt.plot(mlpHinge_history2.history['accuracy'], label='Train Accuracy', color='royalblue')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.legend()
plt.grid()

# loss
plt.subplot(1, 2, 2)  
plt.plot(mlpHinge_history2.history['loss'], label='Train Loss', color='indianred')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

print(f"Response Time : {mlpHinge_rt:.2f}")


K-Nearest Neighbors and Nearest Centroid

In [ ]:
# KNN
for i in [1, 3]:
    knn = KNeighborsClassifier(n_neighbors=i)

    knn_st = time.time()
    knn.fit(xtr_in, ytr)
    knn_et = time.time()
    knn_rt = knn_et - knn_st


    knn_tr_predictions = knn.predict(xtr_in)

    train_accuracy = accuracy_score(ytr, knn_tr_predictions)
    print(f" Train Accuracy using KNN with k={i} : {train_accuracy* 100:.2f}")


    knn_te_predictions = knn.predict(xte_in)

    test_accuracy = accuracy_score(yte, knn_te_predictions)
    print(f" Test Accuracy using KNN with k={i} : {test_accuracy* 100:.2f}")
    print(f"Response Time using KNN with k={i}  : {knn_rt:.8f}")

    knn_cm = confusion_matrix(yte, knn_te_predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=knn_cm, display_labels=classes)
    disp.plot(cmap='Blues', xticks_rotation=90)
    plt.xlabel('Predicted Class')
    plt.ylabel('True Class')
    plt.title(f'Confusion Matrix using KNN with k={i}')
    plt.show()


    
# NC
nc = NearestCentroid()

nc_st = time.time()
nc.fit(xtr_in, ytr)
nc_et = time.time()
nc_rt = nc_et - nc_st


nc_tr_predictions=nc.predict(xtr_in)

train_accuracy = accuracy_score(ytr, nc_tr_predictions)
print(f"Train Accuracy using NC : {train_accuracy* 100:.2f}")


nc_te_predictions=nc.predict(xte_in)

test_accuracy = accuracy_score(yte, nc_te_predictions)
print(f" Test Accuracy using NC : {test_accuracy* 100:.2f}")
print(f"Response Time with NC : {nc_rt:.8f}")

nc_cm = confusion_matrix(yte, nc_te_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=nc_cm, display_labels=classes)
disp.plot(cmap='Blues', xticks_rotation=90) 
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.title(f'Confusion Matrix using NC ')
plt.show()